<a href="https://colab.research.google.com/github/eduswiss/hands-on-algorithmic-trading-with-python/blob/main/notebooks/03_Data_Loading%2C_Storage%2C_and_File_Formats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading, Storage, and File Formats

## Reading and Writing Data in Text Format

In [1]:
import pandas as pd

Mount Google Drive to be able to access data files from Colaboratory Notebooks

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Check the content of the `examples` directory.

In [4]:
!ls '/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples'

csv_mindex.csv	ex2.csv  ex4.csv  ex6.csv  mydata.csv  tseries.csv
ex1.csv		ex3.txt  ex5.csv  ex7.csv  out.csv


Check the content of the example CSV file.

In [5]:
!cat /content/drive/My\ Drive/Colab\ Notebooks/Hands-On\ Algorithmic\ Trading\ with\ Python/examples/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


Use `read_csv` to read it into a DataFrame.

Note that `ex1.csv` have a header row.

In [6]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex1.csv')

In [7]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Load the data from the file without headers.

In [8]:
!cat /content/drive/My\ Drive/Colab\ Notebooks/Hands-On\ Algorithmic\ Trading\ with\ Python/examples/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [9]:
pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Add headers while loading data.

In [10]:
pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex2.csv', 
            names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Define one of the columns to be the index of the returned DataFrame.

In [11]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex2.csv', 
            names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


To form a hierarchical index from multiple columns, pass a list of column numbers or names.

In [12]:
!cat /content/drive/My\ Drive/Colab\ Notebooks/Hands-On\ Algorithmic\ Trading\ with\ Python/examples/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [13]:
parsed = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/csv_mindex.csv', 
                     index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimiter, using whitespace or some other pattern to separate fields.

In [14]:
!cat /content/drive/My\ Drive/Colab\ Notebooks/Hands-On\ Algorithmic\ Trading\ with\ Python/examples/ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491

In [15]:
list(open('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491']

Can pass a regular expression as a delimiter for `read_csv`.

Because there was one fewer column name than the number of data rows, `read_csv` infers that the first column should be the DataFrame’s index in this special case.

In [16]:
result = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex3.txt', 
                     sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


The parser functions have many additional arguments to help handle the wide variety of exception file formats. 

For example, it is possible to skip the first, third, and fourth rows of a file with `skiprows`.

In [17]:
!cat /content/drive/My\ Drive/Colab\ Notebooks/Hands-On\ Algorithmic\ Trading\ with\ Python/examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [18]:
pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex4.csv', 
            skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Handling missing values is an important and frequently nuanced part of the file parsing process. Missing data is usually either not present (empty string) or marked by some sentinel value. By default, pandas uses a set of commonly occurring sentinels, such as `NA` and `NULL`.

In [19]:
!cat /content/drive/My\ Drive/Colab\ Notebooks/Hands-On\ Algorithmic\ Trading\ with\ Python/examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [20]:
result = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex5.csv')
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [21]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


The `na_values` option can take either a list or set of strings to consider missing values.

In [22]:
result = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex5.csv', 
                     na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Different `NA` sentinels can be specified for each column in a dict.

In [23]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex5.csv', 
            na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in Pieces

When processing very large files or figuring out the right set of arguments to correctly process a large file, you may only want to read in a small piece of a file or iterate through smaller chunks of the file.

Before we look at a large file, we make the pandas display settings more compact.

In [24]:
pd.options.display.max_rows = 10

In [25]:
result = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
294,0.354628,-0.133116,0.283763,-0.837063,Q
295,2.311896,-0.417070,-1.409599,-0.515821,L
296,-0.479893,-0.650419,0.745152,-0.646038,E
297,0.523331,0.787112,0.486066,1.093156,K


In [26]:
pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex6.csv', 
            nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [27]:
chunker = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex6.csv', 
                      chunksize=1000)
chunker

In [28]:
tot = pd.Series([])
for piece in chunker:
  tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot[:10]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


G    60.0
L    60.0
Q    30.0
R    30.0
K    30.0
B    30.0
E    30.0
0    29.0
dtype: float64

### Writing Data to Text Format

In [29]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [30]:
data.to_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/out.csv')

In [31]:
!cat '/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/out.csv'

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


Other delimiters can be used.

In [32]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Missing values appear as empty strings in the output. They can be denoted by some other sentinel value.

In [33]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


Writing row and column labels can be disabled.

In [34]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


Only subset of columns can be written, and in the order chosen.

In [35]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series also has a `to_csv` method.

In [36]:
import numpy as np

dates = pd.date_range('1/1/2000', periods=7)
ts = pd.Series(np.arange(7), index=dates)
ts.to_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/tseries.csv')
!cat '/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/tseries.csv'

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


### Working with Delimited Formats

It’s possible to load most forms of tabular data from disk using functions like `pandas.read_csv`. In some cases, however, some manual processing may be necessary. It’s not uncommon to receive a file with one or more malformed lines that trip up `read_csv`.

In [37]:
!cat '/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex7.csv'

"a","b","c"
"1","2","3"
"1","2","3"


For any file with a single-character delimiter, Python’s built-in `csv` module can be used. To use it, pass any open file or file-like object to `csv.reader`:

In [38]:
import csv
f = open('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex7.csv')

reader = csv.reader(f)

Iterating through the reader like a file yields tuples of values with any quote characters removed:

In [39]:
for line in reader:
  print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


Read the file into the list of lines

In [40]:
with open('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex7.csv') as f:
  lines = list(csv.reader(f))

split the lines into the header line and the data lines

In [41]:
header, values = lines[0], lines[1:]

Then can create a dictionary of data columns using a dictionary comprehension and the expression `zip(*values)`, which transposes rows to columns:

In [42]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV files come in many different flavors. To define a new format with a different delimiter, string quoting convention, or line terminator, define a simple subclass of `csv.Dialect`:

In [43]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [44]:
# reader = csv.reader(f, dialect=my_dialect)

It also possible to give individual CSV dialect parameters as keywords to `csv.reader` without having to define a subclass

In [45]:
# reader = csv.reader(f, delimiter='|')

To write delimited files manually, use `csv.writer`. It accepts an open, writable file object and the same dialect and format options as `csv.reader`:

In [46]:
with open('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

In [47]:
!cat '/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/mydata.csv'

one;two;three
1;2;3
4;5;6
7;8;9


### JSON Data

JSON (short for JavaScript Object Notation) has become one of the standard formats for sending data by HTTP request between web browsers and other applications.

In [48]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

`json`  is built into the Python standard library. To convert a JSON string to Python form, use `json.loads`:

In [49]:
import json

result = json.loads(obj)
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 30, 'name': 'Scott', 'pets': ['Zeus', 'Zuko']},
  {'age': 38, 'name': 'Katie', 'pets': ['Sixes', 'Stache', 'Cisco']}]}

`json.dumps`, on the other hand, converts a Python object back to JSON:

In [50]:
asjson = json.dumps(result)

To convert a JSON object or list of objects to a DataFrame or some other data structure for analysis can be done by passing a list of dicts (which were previously JSON objects) to the DataFrame constructor and select a subset of the data fields:

In [51]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


The `pandas.read_json` can automatically convert JSON datasets in specific arrangements into a Series or DataFrame.

In [52]:
!cat '/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/example.json'

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]
 

The default options for `pandas.read_json` assume that each object in the JSON array is a row in the table:

In [53]:
data = pd.read_json('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/example.json')
data

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


To export data from pandas to JSON, one way is to use the `to_json` methods on Series and DataFrame:

In [54]:
print(data.to_json())

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [55]:
print(data.to_json(orient='records'))

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


### XML and HTML: Web Scraping

pandas has a built-in function, `read_html`, which uses libraries like lxml and Beautiful Soup to automatically parse tables out of HTML files as DataFrame objects.

First, need to install some additional libraries used by `read_html`:

In [56]:
!pip install lxml

In [57]:
!pip install beautifulsoup4 html5lib

The `pandas.read_html` function has a number of options, but by default it searches for and attempts to parse all tabular data contained within `<table>` tags. The result is a list of DataFrame objects:

In [58]:
tables = pd.read_html('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/fdic_failed_bank_list.html')
len(tables)

1

In [59]:
failures = tables[0]
failures.head()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date
0,Allied Bank,Mulberry,AR,91,Today's Bank,"September 23, 2016","November 17, 2016"
1,The Woodbury Banking Company,Woodbury,GA,11297,United Bank,"August 19, 2016","November 17, 2016"
2,First CornerStone Bank,King of Prussia,PA,35312,First-Citizens Bank & Trust Company,"May 6, 2016","September 6, 2016"
3,Trust Company Bank,Memphis,TN,9956,The Bank of Fayette County,"April 29, 2016","September 6, 2016"
4,North Milwaukee State Bank,Milwaukee,WI,20364,First-Citizens Bank & Trust Company,"March 11, 2016","June 16, 2016"


Do some data cleaning and analysis, like computing the number of bank failures by year:

In [60]:
close_timestamps = pd.to_datetime(failures['Closing Date'])
close_timestamps.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

#### Parsing XML with lxml.objectify

XML (eXtensible Markup Language) is another common structured data format supporting hierarchical, nested data with metadata.

Here is an example of how to use lxml to parse data from a more general XML format.

The New York Metropolitan Transportation Authority (MTA) publishes a number of data series about its bus and train services. Here we’ll look at the performance data, which is contained in a set of XML files. Each train or bus service has a different file (like *Performance_MNR.xml* for the Metro-North Railroad) containing monthly data as a series of XML records that look like this:

In [61]:
"""<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>"""

'<INDICATOR>\n  <INDICATOR_SEQ>373889</INDICATOR_SEQ>\n  <PARENT_SEQ></PARENT_SEQ>\n  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>\n  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>\n  <DESCRIPTION>Percent of the time that escalators are operational\n  systemwide. The availability rate is based on physical observations performed\n  the morning of regular business days only. This is a new indicator the agency\n  began reporting in 2009.</DESCRIPTION>\n  <PERIOD_YEAR>2011</PERIOD_YEAR>\n  <PERIOD_MONTH>12</PERIOD_MONTH>\n  <CATEGORY>Service Indicators</CATEGORY>\n  <FREQUENCY>M</FREQUENCY>\n  <DESIRED_CHANGE>U</DESIRED_CHANGE>\n  <INDICATOR_UNIT>%</INDICATOR_UNIT>\n  <DECIMAL_PLACES>1</DECIMAL_PLACES>\n  <YTD_TARGET>97.00</YTD_TARGET>\n  <YTD_ACTUAL></YTD_ACTUAL>\n  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>\n  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>\n</INDICATOR>'

Using `lxml.objectify`, can parse the file and get a reference to the root node of the XML file with `getroot`:

In [62]:
from lxml import objectify

path = '/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

`root.INDICATOR` returns a generator yielding each `<INDICATOR>` XML element. For each record, we can populate a dict of tag names (like `YTD_ACTUAL`) to data values (excluding a few tags):

In [63]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
  el_data = {}
  for child in elt.getchildren():
    if child.tag in skip_fields:
      continue
    el_data[child.tag] = child.pyval
  data.append(el_data)

Lastly, convert this list of dicts into a DataFrame:

In [64]:
perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95,96.9,95,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95,96,95,95
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95,96.3,95,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95,96.8,95,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95,96.6,95,95.8


XML data can get much more complicated than this example. Each tag can have metadata, too. Consider an HTML link tag, which is also valid XML:

In [65]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

Now it is possible to access any of the fields (like `href`) in the tag or the link text:

In [66]:
root

<Element a at 0x7f06b43f7ac8>

In [67]:
root.get('href')

'http://www.google.com'

In [68]:
root.text

'Google'

## Binary Data Formats

One of the easiest ways to store data (also known as *serialization*) efficiently in binary format is using Python’s built-in `pickle` serialization. pandas objects all have a `to_pickle` method that writes the data to disk in pickle format:

In [69]:
frame = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [70]:
frame.to_pickle('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/frame_pickle')

Any “pickled” object stored in a file can be read by using the built-in `pickle` directly, or even more conveniently using `pandas.read_pickle`:

In [71]:
pd.read_pickle('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


**CAUTION**

`pickle` is only recommended as a short-term storage format. The problem is that it is hard to guarantee that the format will be stable over time; an object pickled today may not unpickle with a later version of a library.

### Using HDF5 Format

HDF5 is a well-regarded file format intended for storing large quantities of scientific array data. It is available as a C library, and it has interfaces available in many other languages, including Java, Julia, MATLAB, and Python. The “HDF” in HDF5 stands for *hierarchical data format*. Each HDF5 file can store multiple datasets and supporting metadata. Compared with simpler formats, HDF5 supports on-the-fly compression with a variety of compression modes, enabling data with repeated patterns to be stored more efficiently. HDF5 can be a good choice for working with very large datasets that don’t fit into memory, as you can efficiently read and write small sections of much larger arrays.

While it’s possible to directly access HDF5 files using either the PyTables or h5py libraries, pandas provides a high-level interface that simplifies storing Series and DataFrame object. The `HDFStore` class works like a dict and handles the low-level details:

In [74]:
frame = pd.DataFrame({'a': np.random.randn(100)})

store = pd.HDFStore('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: /content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/mydata.h5

Objects contained in the HDF5 file can then be retrieved with the same dict-like API:

In [75]:
store['obj1']

,a
0,-1.316964
1,-0.180900
2,-0.515896
3,-1.615075
4,-0.627694
...,...
95,-2.345725
96,-0.991933
97,-1.004936
98,0.440216


`HDFStore` supports two storage schemas, `'fixed'` and `'table'`. The latter is generally slower, but it supports query operations using a special syntax:

In [76]:
store.put('obj2', frame, format='table')

store.select('obj2', where=['index >= 10 and index <= 15'])

,a
10,0.688173
11,0.301662
12,0.037536
13,-0.661110
14,0.345365
15,0.552141


In [77]:
store.close()

The put is an explicit version of the `store['obj2'] = frame` method but allows to set other options like the storage format.

The `pandas.read_hdf` function gives a shortcut to these tools:

In [78]:
frame.to_hdf('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/mydata.h5', 
             'obj3', 
             format='table')

pd.read_hdf('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/mydata.h5', 
            'obj3',
            where=['index < 5'])

,a
0,-1.316964
1,-0.180900
2,-0.515896
3,-1.615075
4,-0.627694


**NOTE**

When processing data that is stored on remote servers, like Amazon S3 or HDFS, using a different binary format designed for distributed storage like Apache Parquet may be more suitable.

**CAUTION**

HDF5 is *not* a database. It is best suited for write-once, read-many datasets. While data can be added to a file at any time, if multiple writers do so simultaneously, the file can become corrupted.

### Reading Microsoft Excel Files

pandas also supports reading tabular data stored in Excel 2003 (and higher) files using either the `ExcelFile` class or `pandas.read_excel` function. Internally these tools use the add-on packages `xlrd` and `openpyxl` to read XLS and XLSX files, respectively. 

To use `ExcelFile`, create an instance by passing a path to an `xls` or `xlsx` file:

In [79]:
xlsx = pd.ExcelFile('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex1.xlsx')

Data stored in a sheet can then be read into DataFrame with `parse`:

In [80]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


When reading multiple sheets in a file, then it is faster to create the `ExcelFile`, but it's also possible just simply pass the filename to `pandas.read_excel`:

In [83]:
frame = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex1.xlsx', 
                      'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


To write pandas data to Excel format, must first create an `ExcelWriter`, then write data to it using pandas objects’ `to_excel` method:

In [84]:
writer = pd.ExcelWriter('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

It is also possible to pass a file path to `to_excel` and avoid the `ExcelWriter`:

In [85]:
frame.to_excel('/content/drive/My Drive/Colab Notebooks/Hands-On Algorithmic Trading with Python/examples/ex2.xlsx')

## Interacting with Web APIs

Many websites have public APIs providing data feeds via JSON or some other format. There are a number of ways to access these APIs from Python; one easy-to-use method is the `requests` package.

To find the last 30 GitHub issues for pandas on GitHub, can make a `GET` HTTP request using the add-on `requests` library:

In [86]:
import requests

url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

The Response object’s `json` method will return a dictionary containing JSON parsed into native Python objects:

In [88]:
data = resp.json()
data[0]['title']

'BUG: InvalidIndexError when doing fillna with index that is repeated in df but not in fillna argument'

Each element in `data` is a dictionary containing all of the data found on a GitHub issue page (except for the comments). It is possible to pass `data` directly to DataFrame and extract fields of interest:

In [89]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])
issues

,number,title,labels,state
0,36974,BUG: InvalidIndexError when doing fillna with ...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
1,36973,BUG: merge with MultiIndex including a Categor...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
2,36972,BUG: warning when using colors 'CN',"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
3,36971,fix issue 36970,[],open
4,36970,DOC: improve description of the examplewhich d...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
...,...,...,...,...
25,36943,REF/TYP: define methods non-dynamically for Sp...,[],open
26,36942,Test astyp via loc to int64,[],open
27,36941,BUG: series.eq(other) does not equal series ==...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
28,36940,REGR: DateOffset attributes update fails with ...,"[{'id': 32815646, 'node_id': 'MDU6TGFiZWwzMjgx...",open


## Interacting with Databases

In a business setting, most data may not be stored in text or Excel files. SQL-based relational databases (such as SQL Server, PostgreSQL, and MySQL) are in wide use, and many alternative databases have become quite popular. The choice of database is usually dependent on the performance, data integrity, and scalability needs of an application.

Loading data from SQL into a DataFrame is fairly straightforward, and pandas has some functions to simplify the process. As an example, a SQLite database will be created using Python’s built-in `sqlite3` driver:

In [98]:
import sqlite3

query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL,        d INTEGER
);"""

con = sqlite3.connect('mydata.sqlite')

con.execute(query)
con.commit()

Then, insert a few rows of data:

In [99]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

Most Python SQL drivers (PyODBC, psycopg2, MySQLdb, pymssql, etc.) return a list of tuples when selecting data from a table:

In [100]:
cursor = con.execute('select * from test')

rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

It is possible to pass the list of tuples to the DataFrame constructor, but also needs the column names, contained in the cursor’s `description` attribute:

In [101]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [102]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


The SQLAlchemy project is a popular Python SQL toolkit that abstracts away many of the common differences between SQL databases. pandas has a `read_sql` function that enables to read data easily from a general SQLAlchemy connection. Here's how to connect to the same SQLite database with SQLAlchemy and read data from the table created before:

In [104]:
import sqlalchemy as sqla

db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
